In [ ]:
# !pip install openpyxl
from openpyxl import load_workbook

In [ ]:
# !pip install numpy
# !pip install pandas
import numpy as np
import pandas as pd

In [ ]:
# loading workbook and accessing NRs as lists
a = load_workbook("Template/template 1.0.0.xlsx")

In [6]:
# accessing NRs as lists
all_NRs = a.defined_names
list_NRs = list(a.defined_names.keys())

In [9]:
# Iterating over NRs and accessing their attributes, storing in dict
dict_of_cell_references = {}
for NR in list_NRs:
    dict_of_cell_references[NR] = all_NRs[NR].attr_text.split("!")[1]

In [11]:
def letter_to_number(col):
    col = col.upper()
    n = 0
    for c in col:
        if not ("A" <= c <= "Z"):
            raise ValueError(f"Invalid column letter: {col}")
        n = n * 26 + (ord(c) - ord("A") + 1)
    return n

# examples
letter_to_number("A")   # 1

1

In [ ]:
# Creating shapes of the cell references from ranges
dict_of_cell_shapes = {}
for i in dict_of_cell_references.keys():
    a = dict_of_cell_references[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    dict_of_cell_shapes[i] = a

dict_of_cell_shapes

In [ ]:
df = pd.DataFrame({
    "name_ranges": list_NRs,
    "cell_references": list(dict_of_cell_references.values()),
    "cell_shapes": list(dict_of_cell_shapes.values()),
})
df

In [ ]:
# condition for seeing whether cell reference is a formula
a["Social Disclosures"].cell(row=46, column=4).data_type == "f"  # True

In [ ]:
n = 0
for i in range(1, 10):
    print(n)
    print(type(n))
    n += 1

In [105]:
with_data = load_workbook("with_data.xlsx")
empty = load_workbook("empty.xlsx")
NR_old = with_data.defined_names
list_NR_old = list(with_data.defined_names.keys())
NR_new = empty.defined_names
list_NR_new = list(empty.defined_names.keys())

In [106]:
references_old = []
for NR in NR_old:
    references_old.append(NR_old[NR].attr_text.split("!")[1])

shapes_old = []
for i in range(len(list_NR_old)):
    a = references_old[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    a = [int(ciao) for ciao in a]
    shapes_old.append(a)

df_old = pd.DataFrame({
    "name_ranges": list_NR_old,
    "cell_references": references_old,
    "cell_shapes": shapes_old,
})
df_old

,name_ranges,cell_references,cell_shapes
0,number,$D$3,"[4, 3]"
1,stringa,$F$3:$F$6,"[6, 3, 6, 6]"


In [107]:
# Extracting cell values from old workbook based on cell references and shapes

sheet_old = with_data["Sheet1"]

cell_values = []

for i in range(len(df_old)):
    if len(df_old["cell_shapes"][i]) == 2:
        cell_values.append([sheet_old[df_old["cell_references"][i]].value])
    else:
        cell_values_temp = []
        for row in sheet_old.iter_rows( min_row=df_old["cell_shapes"][i][1],
                                        max_row=df_old["cell_shapes"][i][3],
                                        min_col=df_old["cell_shapes"][i][0],
                                        max_col=df_old["cell_shapes"][i][2] ):
            cell_values_temp.append([])
            for cell in row:
                cell_values_temp[-1].append(cell.value)
        cell_values.append(cell_values_temp)

df_old["cell_values"] = cell_values
df_old


,name_ranges,cell_references,cell_shapes,cell_values
0,number,$D$3,"[4, 3]",[56]
1,stringa,$F$3:$F$6,"[6, 3, 6, 6]","[[ciao], [come], [va], [ciao]]"


In [108]:
references_new = []
for NR in NR_new:
    references_new.append(NR_new[NR].attr_text.split("!")[1])

shapes_new = []
for i in range(len(list_NR_new)):
    a = references_new[i].split("$")[1:]
    for j in range(len(a)):
        if j % 2 != 1:
            a[j] = letter_to_number(a[j])
    if( len(a) == 4 ):
        a[1] = a[1][:-1]
    a = [int(ciao) for ciao in a]
    shapes_new.append(a)

df_new = pd.DataFrame({
    "name_ranges": list_NR_new,
    "cell_references": references_new,
    "cell_shapes": shapes_new,
})
df_new

,name_ranges,cell_references,cell_shapes
0,number,$C$7,"[3, 7]"
1,stringa,$H$8:$H$11,"[8, 8, 8, 11]"


In [109]:
# merging values from old to new (within dataframes)

df_old_tomerge = df_old.iloc[:,[0,3]]
df_new_withvalues = pd.merge(df_new, df_old_tomerge)
df_new_withvalues


,name_ranges,cell_references,cell_shapes,cell_values
0,number,$C$7,"[3, 7]",[56]
1,stringa,$H$8:$H$11,"[8, 8, 8, 11]","[[ciao], [come], [va], [ciao]]"


In [113]:
# paste values into new workbook

sheet_new = empty["Sheet1"]

for i in range(len(df_new_withvalues)):
    if len(df_new_withvalues["cell_shapes"][i]) == 2:
        sheet_new[df_new_withvalues["cell_references"][i]].value = df_new_withvalues["cell_values"][i][0]
    else:
        for row in range( df_new_withvalues["cell_shapes"][i][1],
                                df_new_withvalues["cell_shapes"][i][3] + 1 ):
            for col in range( df_new_withvalues["cell_shapes"][i][0],
                              df_new_withvalues["cell_shapes"][i][2] + 1 ):
                sheet_new.cell(row=row, column=col).value = df_new_withvalues["cell_values"][i][row - df_new_withvalues["cell_shapes"][i][1]][col - df_new_withvalues["cell_shapes"][i][0]]

empty.save("new_with_data.xlsx")